# Subword Tokenizer

| **Tokenizer 방식** | **토큰 단위**                      | **vocab size** | **미등록 단어에 대한 가정**                                                                                  |
|---------------------|------------------------------------|----------------|-------------------------------------------------------------------------------------------------------------|
| **사전 기반**       | 알려진 단어/형태소의 결합           | unlimited       | - 알려진 단어/형태소의 결합이라고 가정<br>- 필요한 형태소 분석 가능<br>- 사전에 등록되지 않은 단어는 UNK 처리 |
| **sub-word**        | 알려진 글자 및 sub-word            | fixed           | - 알려진 sub-words로 분해<br>- 예: appear → app + ear<br>- 자주 등장하는 단어를 제대로 인식 가능<br>- UNK의 개수 최소화 |

### 네이버 영화 리뷰 학습

In [1]:
# 네이버 영화 리뷰 데이터 다운로드(캐시 저장) 함수
import urllib.request                                                               # URL에서 파일을 다운로드하는 모듈
import os                                                                           # 경로/폴더 생성 등 파일 시스템 처리 모듈
# 파일 다운로드 함수 : 지정한 URL(origin)의 파일을 토컬 캐시에 저장하고 경로를 반환
def get_file(filename, origin):
    cache_dir = os.path.expanduser('~/.torch/datasets')                             # 캐시 디렉토리 경로 생성
    os.makedirs(cache_dir, exist_ok=True)                                           # 캐시 폴더 없으면 생성
    filepath = os.path.join(cache_dir, filename)                                    # 저장할 파일 전체 파일 경로
    
    if not os.path.exists(filepath):                                                 # 파일이 없으면
        print(f'다운로드 진행중! {origin}')
        urllib.request.urlretrieve(origin, filepath)                                # origin에서 파일을 내려받아 filepath에 저장
        
    return filepath                                                                 # 로컬에 저장한 파일 경로 반환

In [2]:
# NSMC(네이버 영화 리뷰) 학습/테스트 파일 다운로드 경로 확인
ratings_train_path = get_file(
    'ratings_train.txt',
    'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt'
)
ratings_test_path = get_file(
    'ratings_test.txt',
    'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt'
)

ratings_train_path, ratings_test_path

('C:\\Users\\Playdata/.torch/datasets\\ratings_train.txt',
 'C:\\Users\\Playdata/.torch/datasets\\ratings_test.txt')

In [3]:
# NSMC 데이터 텍스트 파일을 DataFrame 으로 로드 후 확인
import pandas as pd

ratings_train_df = pd.read_csv(ratings_train_path, sep='\t')             # 학습 데이터 (탭 구분)
ratings_test_df = pd.read_csv(ratings_test_path, sep='\t')               # 테스트 데이터 (탭 구분)

display(ratings_train_df)
display(ratings_test_df)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


.txt 파일이 id \t document \t label 구조로 되어있다.
결국 테이블 형태로 이루어져있어 read_csv로 읽어올 수 있다.

In [4]:
print(ratings_train_df.isnull().sum())
print(ratings_test_df.isnull().sum())

id          0
document    5
label       0
dtype: int64
id          0
document    3
label       0
dtype: int64


In [5]:
ratings_train_df = ratings_train_df.dropna(how='any')
ratings_test_df = ratings_test_df.dropna(how='any')

ratings_train_df.shape, ratings_test_df.shape

((149995, 3), (49997, 3))

In [6]:
# 학습 리뷰 문장을 텍스트 파일로 저장
with open('naver_review.txt', 'w', encoding='utf-8') as f:      # naver_review.txt를 쓰기 모드로 열기
    for doc in ratings_train_df['document'].values:             # 학습 데이터의 리뷰 문장을 순회
        f.write(doc + '\n')                                     # 각 문장을 한 줄씩 파일에 기록

### SentencePieceTokenizer

In [7]:
! pip install sentencepiece

In [8]:
import sentencepiece as spt     # sentencePiece(서브워드 토크나이저) 라이브러리

input = 'naver_review.txt'      # 학습에 사용할 텍스트 파일
vocab_size = 10000              # 만들 서브워드 사전 크기 (토큰 개수)
model_prefix = 'naver_review'   # 저장할 모델/사전 파일 일므 접두어(naver_review.model, naver_review.vocab)

cmd = f'--input={input} --model_prefix={model_prefix} --vocab_size={vocab_size}'    # 학습 옵션 문자열

spt.SentencePieceTrainer.Train(cmd) # SentencePiece 모델 학습 후 파일로 저장


In [9]:
# 학습된 SentencePiece 토크나이저 로드 후 토큰화/시퀀스 변환 확인
sp = spt.SentencePieceProcessor()                   # SentencePiece 토크나이저(Processor) 객체 생성
sp.Load(f'{model_prefix}.model')                    # 학습된 토크나이저 모델(.model) 로드

for doc in ratings_train_df['document'].values[:3]: # 리뷰 맨 앞 3개
    print(doc)                                      # 원문 출력
    print(sp.encode_as_pieces(doc))                 # 서브워드 단위로 토큰화된 결과 출력(문자열 토큰 리스트)
    print(sp.encode_as_ids(doc))                    # 토큰을 정수 ID로 변환한 시퀀스 출력
    print()

아 더빙.. 진짜 짜증나네요 목소리
['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']
[62, 877, 5, 31, 2019, 68, 1710]

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['▁흠', '...', '포스터', '보고', '▁초딩', '영화', '줄', '....', '오', '버', '연기', '조차', '▁가볍지', '▁않', '구나']
[1634, 8, 4908, 159, 1460, 33, 264, 60, 173, 548, 410, 1224, 7396, 754, 440]

너무재밓었다그래서보는것을추천한다
['▁너무', '재', '밓', '었다', '그래서', '보', '는것을', '추천', '한다']
[23, 369, 9781, 429, 3780, 143, 6266, 1945, 314]



In [10]:
# SentencePiece 어휘 크기(vocabulary size) 확인
# sp.get_piece_size()   # 현재 로드한 SentencePiece 모델의 토큰(피스) 개수
sp.GetPieceSize()       # 위와 동일

10000

In [17]:
# SentencePiece 인코딩(토큰/ID) + 디코딩(복원)
# 인코딩
text = ratings_test_df['document'][100]                 # 테스트 데이터의 101번째 리뷰
tokens = sp.encode_as_pieces(text)                      # 텍스트 -> 서브워드(piece) 토큰 리스트로 변환
id_tokens = sp.encode_as_ids(text)                      # 텍스트 -> 서브워드 토큰 -> 정수 ID 시퀀스로 변환

print(text)
print(tokens)
print(id_tokens)

print("".join(tokens).replace("\u2581", " ").strip())        # 토큰을 붙인 뒤, _(공백 표시)를 실제 공백으로 바꿈

# 디코딩
print(sp.decode_pieces(tokens))
print(sp.decode_ids(id_tokens))

space_token = "\u2581"
print(space_token)

걸작은 몇안되고 졸작들만 넘쳐난다.
['▁걸작', '은', '▁몇', '안되고', '▁졸작', '들만', '▁넘', '쳐', '난다', '.']
[1060, 18, 621, 6979, 728, 3291, 165, 705, 1003, 4]
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.
▁


### BertWordPieceTokenizer

In [ ]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(
    lowercase = False,                          # 대소문자의 소문자화 비활성화 (영어면 설정)
    strip_accents = False                       # 악센트 제거 비활성화 (불필요한 변형/분리 방지)
)
vocab_size = 10000                              # 만들 vocab 개수

tokenizer.train(                                
    files = ['naver_review.txt'],               # 학습에 사용할 파일
    vocab_size = vocab_size,                    # 어휘 크기 (토큰 개수)
    min_frequency = 5,                          # 최소 5회 이상 등장한 토큰만 vcoab 후보로 포함
    show_progress = True                        # 학습 진행 상황 출력
)

In [13]:
# 학습된 WordPiece 토크나이저 vocab 파일 저장
tokenizer.save_model('./', 'bert_word_piece_from_naver_review')

['./bert_word_piece_from_naver_review-vocab.txt']

In [14]:
# Word_iece 토큰화(인코딩) + 디커딩(복원)
text = ratings_test_df['document'][100]

# 인코딩
encoded = tokenizer.encode(text)                # 텍스트를 WordPiece 방식으로 토큰화하고, 토큰/ID로 인코딩

print(encoded.tokens)                           # WordPiece 토큰 리스트 출력 (서브워드 포함)
print(encoded.ids)                              # 토큰에 대응하는 정수 ID 시퀀스 출력

# 디코딩
print(text)
print(tokenizer.decode(encoded.ids))            # ID 시퀀스를 다시 텍스트로 디코딩해 확인

['걸작', '##은', '몇', '##안되고', '졸작', '##들만', '넘쳐', '##난다', '.']
[2759, 1018, 444, 9504, 2589, 3799, 8337, 2430, 16]
걸작은 몇안되고 졸작들만 넘쳐난다.
걸작은 몇안되고 졸작들만 넘쳐난다.


- 여태까지 진행한 사항은 **네이버 영화리뷰 텍스트를 모델이 먹을 수 있는 숫자 시퀀스로 바꾸는 ‘토크나이저(서브워드 사전)’를 직접 만들고, 인코딩/디코딩이 되는지 확인**한 것임.

크게 목적은 3가지이다.

1. **단어사전(vocab) 만들기**
    - 리뷰 전체를 보고 자주 나오는 글자/부분단어(서브워드)를 모아 **토큰 사전**을 만든다.
    - OOV(처음 보는 단어) 문제를 줄이기 위해 “서브워드” 단위로 쪼개는 방식을 사용.
2. **텍스트 → 숫자(IDs)로 변환**
    - `encode_as_ids()`(SentencePiece) / `tokenizer.encode().ids`(WordPiece)로 문장을 **정수 ID 시퀀스**로 바꿔서, 이후 **Embedding/RNN/Transformer** 같은 모델 입력으로 넣을 수 있게 함.
3. **디코딩으로 검증(정상 동작 확인)**
    - `decode_ids()` / `decode()`로 다시 문장으로 복원해 보면서 “토큰화가 제대로 되는지”, “공백/특수문자 처리 문제가 없는지”를 확인.

추가로, 이번에 **SentencePiece vs WordPiece**를 둘 다 해본 건:

- 같은 한국어 데이터에서도 토큰화 방식이 어떻게 달라지는지 비교하고,
- 나중에 BERT류(WordPiece)나 일반 서브워드 모델(SentencePiece)에 맞게 선택하려는 목적이다.

- **표준형 딥러닝 텍스트 파이프라인**
    - **텍스트**: 원문 문장/문서(모델이 직접 처리 못 하는 문자열)
    - **정규화/전처리**: 노이즈 감소(소문자화, 특수문자 처리, 공백 정리 등)로 입력 형태를 일관되게 만듦
    - **토큰화(SentencePiece/WordPiece 등)**: 문장을 단어/서브워드 단위로 쪼개 OOV를 줄이고 모델이 다룰 “토큰”을 만듦
    - **ID 시퀀스**: 토큰을 정수로 매핑해 모델 입력 가능한 숫자 시퀀스로 변환(= vocab 기반 인덱싱)
    - **padding/truncation**: 배치 학습을 위해 길이를 고정(maxlen)하고, 짧으면 채우고 길면 잘라냄
    - **(Embedding)**: 정수 ID를 저차원 실수 벡터로 변환해 의미/유사도 학습이 가능하게 함(dense representation)
    - **Encoder(RNN/CNN/Transformer)**: 시퀀스에서 문맥/패턴을 추출해 문장 표현(특징 벡터)을 생성
    - **출력층**: 목적에 맞게 예측(분류 확률, 회귀값, 다음 토큰 등)을 계산하는 마지막 레이어

| 구분 | 핵심 목표 | 무엇을 했나(작업) | 왜 필요한가(효과) | 대표 함수/형태 | 출력/산출물 |
|---|---|---|---|---|---|
| 진행 요약(한 줄) | **네이버 영화리뷰 텍스트 → 모델 입력 가능한 숫자 시퀀스**로 변환 | **서브워드 토크나이저 사전 직접 학습 → 인코딩/디코딩 검증** | 문자열은 모델이 직접 처리 불가 → 숫자 시퀀스로 바꿔야 Embedding/RNN/Transformer 입력 가능 | SentencePiece/WordPiece 토크나이저 학습·테스트 | vocab(토큰 사전), encode 결과 ID 시퀀스, decode 복원 결과 |
| 목적 1: vocab 만들기 | **토큰 사전(vocab) 구축** | 리뷰 코퍼스에서 자주 나오는 **서브워드(부분단어)** 를 수집/학습 | OOV(처음 보는 단어) 감소, 한국어의 다양한 형태 변형 대응 | SentencePiece: `spm.SentencePieceTrainer...` / WordPiece: `BertWordPieceTokenizer.train(...)` | `vocab` 파일(s), 토큰↔ID 매핑 |
| 목적 2: 텍스트→ID 변환 | 문장을 **정수 ID 시퀀스**로 변환 | 토큰화 후 토큰을 vocab 인덱스로 매핑 | 딥러닝 입력은 숫자 텐서가 필요 | SentencePiece: `encode_as_ids()` / WordPiece: `tokenizer.encode().ids` | 예: `[101, 2345, 67, ...]` (정수 리스트) |
| 목적 3: 디코딩 검증 | 토큰화/인코딩이 정상인지 확인 | ID → 토큰 → 문자열로 복원 테스트 | 공백/특수문자/분절 품질 문제를 조기에 발견 | SentencePiece: `decode_ids()` / WordPiece: `decode()` | 복원 문자열(원문과 유사해야 정상) |
| SentencePiece vs WordPiece 비교 목적 | 같은 한국어 데이터에서 **토큰화 차이** 이해 + 모델 선택 대비 | 두 방식 모두 실험해서 분절 결과/공백 처리/OOV 대응 비교 | BERT류(WordPiece) vs 일반 서브워드 모델(SentencePiece) 선택 근거 확보 | SentencePiece(▁ 공백표시) / WordPiece(## 접두 서브워드) | 토큰 분절 예시, vocab 구성 차이, 성능/편의 비교 인사이트 |

---

## 표준형 딥러닝 텍스트 파이프라인(단계별 상세)

| 단계 | 입력 | 처리(무엇을 함) | 출력 | 핵심 포인트/주의 | 대표 예시 |
|---|---|---|---|---|---|
| 1. 원문 텍스트 | 문자열(리뷰/문장) | 모델이 직접 못 먹는 **자연어 원문** | 텍스트 그대로 | 인코딩(UTF-8), 줄바꿈/이모지 등 노이즈 존재 | `"스토리 너무 좋았음 ㅠㅠ"` |
| 2. 정규화/전처리 | 텍스트 | 공백 정리, 특수문자 처리, (선택) 소문자화/숫자 규칙화 등 | 정규화된 텍스트 | 과도한 정제는 의미 손실 가능(“ㅠㅠ”, “ㅋㅋ” 등) | `"스토리 너무 좋았음"` |
| 3. 토큰화(서브워드) | 정규화 텍스트 | SentencePiece/WordPiece로 문장을 **서브워드 단위**로 분절 | 토큰 시퀀스 | 한국어는 형태 변화가 많아 서브워드가 OOV에 유리 | `["▁스토리", "▁너무", "▁좋", "았", "음"]` 등 |
| 4. ID 시퀀스 변환 | 토큰 시퀀스 | 토큰을 vocab 인덱스로 매핑 | 정수 ID 시퀀스 | 모델 입력은 숫자여야 함. `<unk>`, `<pad>` 같은 특수 토큰 체계 확인 | `[35, 912, 77, 21, 9]` |
| 5. padding / truncation | ID 시퀀스 | 배치 학습을 위해 길이를 `maxlen`으로 고정(짧으면 pad, 길면 절단) | `(batch, timesteps)` 텐서 | truncation 정책(pre/post), pad 토큰 ID 일관성 중요 | `maxlen=50`로 맞춤 |
| 6. Embedding | 정수 텐서 | `Embedding(vocab_size, d)`로 ID를 **d차원 실수 벡터**로 매핑 | `(batch, timesteps, d)` | 원-핫 대신 dense로 효율↑, 의미/유사도 학습 가능 | `d=128` |
| 7. Encoder | 임베딩 시퀀스 | RNN/CNN/Transformer로 문맥·패턴 추출 | 문장 표현(특징) | RNN은 순차, Transformer는 self-attention 기반 문맥학습 | `LSTM/GRU/TransformerEncoder` |
| 8. 출력층(Head) | 특징 벡터 | 목적에 맞는 예측 계산(분류/회귀/생성) | 확률/값/토큰 | 분류는 sigmoid/softmax, 회귀는 linear 등 | 이진분류: `Dense(1, sigmoid)` |

---

## 보너스: 토큰 표기(자주 헷갈리는 것)

| 표기 | 어디서 주로 보나 | 의미 |
|---|---|---|
| `▁` | SentencePiece | **공백(space) 표시**. 토큰 앞에 공백이 있었음을 나타냄 |
| `##` | WordPiece(BERT류) | **서브워드 접두**. 단어 중간에 이어붙는 조각임을 의미 |
| `<unk>` | 공통 | vocab에 없는 토큰(OOV)을 대체하는 특수 토큰 |
| `<pad>` | 공통 | padding에 사용하는 특수 토큰(길이 맞춤) |